In [ ]:
import pandas as pd

In [ ]:
df_goodreads = pd.read_csv('../data/raw/goodreads_best_books_ever.csv')
df_openlibrary = pd.read_csv('../data/raw/openlibrary_fiction_10k.csv')
df_openlibrary_ratings = pd.read_csv('../data/raw/ol_dump_ratings_2025-03-02.tsv', sep='\t', names=['key', 'lending_edition?', 'rating', 'date'])
df_nytimes = pd.read_csv('../data/raw/nytimes_fiction_2015_2025.csv')

In [ ]:
# print(df_nytimes.shape)
# df_nytimes.head(10)

In [ ]:
print(df_goodreads.shape)
df_goodreads.head(10)


In [ ]:
df_goodreads["author"].value_counts()

In [ ]:
display(df_openlibrary.head(10))
display(df_openlibrary.shape)
print()
display(df_openlibrary_ratings.head(10))
display(df_openlibrary_ratings.shape)

In [ ]:
df_goodreads.shape

In [ ]:
df_goodreads.drop(columns=["id", "url"], inplace=True)

In [ ]:
df_goodreads.head(10)

In [ ]:
df_openlibrary_merge = df_openlibrary.merge(df_openlibrary_ratings, how='left', on="key")


In [ ]:
display(df_openlibrary_merge.head(5))
display(df_openlibrary_merge.shape)

In [ ]:
df_openlibrary_merge.dtypes

In [ ]:
import ast

df_openlibrary_merge["author"] = df_openlibrary_merge["authors"].apply(lambda x: ast.literal_eval(x)[0]['name'] if len(ast.literal_eval(x)) > 0 else None)

df_openlibrary_filter = df_openlibrary_merge[["title", "author", "first_publish_year", "rating"]]
df_openlibrary_filter.drop_duplicates(subset="title", inplace=True)

In [ ]:
df_merge = df_openlibrary_filter.merge(df_goodreads, how='left', on='title')
df_merge

In [ ]:
df_merge['rating_y'].isna().value_counts()

In [ ]:
df_merge.dropna(subset=['rating_x', 'rating_y'], how='all', inplace=True)
df_merge

In [ ]:
df_merge['rating_y'] = df_merge['rating_y'].fillna(df_merge['rating_x'])
df_merge.rename(columns={"rating_y": "rating", "author_x": "author"}, inplace=True)
df_merge

In [ ]:
author_counts = df_merge["author"].value_counts()
df_merge["author_encoded"] = df_merge["author"].map(author_counts)

In [ ]:
df_merge.columns

In [ ]:
df_merge = df_merge[['title', 'author', 'first_publish_year',
       'rating', 'author_encoded']]

In [ ]:
df_merge.reset_index(drop=True, inplace=True)
df_merge.dropna(how='any', inplace=True)
df_merge

In [ ]:
df_merge = df_merge[(df_merge["rating"] >= 3.5) & (df_merge["first_publish_year"] >= 1900)]
df_merge

In [ ]:
list(df_merge['title'].values)

In [ ]:
df_merge.to_csv("../data/clean/df_books_clean.csv", index=False)